# Loop 9 Strategic Analysis

## Key Question: Why are we stuck at 70.734327?

After 9 experiments, all achieving the EXACT SAME score, we need to understand:
1. Is the baseline truly at a global optimum?
2. What techniques haven't we tried?
3. What do the top kernels do differently?

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

# Tree shape
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_polygon(cx, cy, deg):
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    x = TX * c - TY * s + cx
    y = TX * s + TY * c + cy
    return x, y

def score_n(df, n):
    rows = df[df['id'].str.startswith(f'{n:03d}_')]
    if len(rows) == 0:
        return float('inf')
    
    all_x, all_y = [], []
    for _, row in rows.iterrows():
        x_val = float(str(row['x']).replace('s', ''))
        y_val = float(str(row['y']).replace('s', ''))
        deg = float(str(row['deg']).replace('s', ''))
        px, py = get_tree_polygon(x_val, y_val, deg)
        all_x.extend(px)
        all_y.extend(py)
    
    side = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
    return side * side / n

print("Analysis of baseline vs overlapping configurations")

In [ ]:
# Load baseline and overlapping CSV
baseline_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv')
overlap_df = pd.read_csv('/home/code/experiments/submission_v21.csv')

# Compare scores for each N
print("Per-N Score Comparison: Baseline vs Overlapping")
print("="*70)

baseline_total = 0
overlap_total = 0
gap_by_n = []

for n in range(1, 201):
    b_score = score_n(baseline_df, n)
    o_score = score_n(overlap_df, n)
    baseline_total += b_score
    overlap_total += o_score
    gap = b_score - o_score
    gap_by_n.append((n, b_score, o_score, gap))

print(f"\nBaseline total: {baseline_total:.6f}")
print(f"Overlap total:  {overlap_total:.6f}")
print(f"Gap:            {baseline_total - overlap_total:.6f}")
print(f"Target:         68.931058")
print(f"Gap to target:  {baseline_total - 68.931058:.6f}")

In [ ]:
# Find N values with biggest gaps
gap_by_n.sort(key=lambda x: -x[3])  # Sort by gap descending

print("\nTop 20 N values with biggest improvement potential:")
print("="*70)
print(f"{'N':>4} {'Baseline':>12} {'Overlap':>12} {'Gap':>12} {'% of Total Gap':>15}")
print("-"*70)

total_gap = baseline_total - overlap_total
for n, b, o, gap in gap_by_n[:20]:
    pct = gap / total_gap * 100 if total_gap > 0 else 0
    print(f"{n:>4} {b:>12.6f} {o:>12.6f} {gap:>12.6f} {pct:>14.1f}%")

In [ ]:
# Analyze where the improvements come from
print("\nScore breakdown by N range:")
print("="*70)

ranges = [(1, 10), (11, 20), (21, 30), (31, 50), (51, 100), (101, 200)]

for start, end in ranges:
    b_sum = sum(x[1] for x in gap_by_n if start <= x[0] <= end)
    o_sum = sum(x[2] for x in gap_by_n if start <= x[0] <= end)
    gap_sum = b_sum - o_sum
    pct = gap_sum / total_gap * 100 if total_gap > 0 else 0
    print(f"N={start:>3}-{end:>3}: Baseline={b_sum:>8.4f}, Overlap={o_sum:>8.4f}, Gap={gap_sum:>8.4f} ({pct:>5.1f}%)")

In [ ]:
# Key insight: The gap is concentrated in small N values
# Let's see what the overlapping configs look like for the top N values

print("\nAnalysis of overlapping configurations for top gap N values:")
print("="*70)

from numba import njit

@njit
def pip_numba(px, py, poly_x, poly_y):
    n = len(poly_x)
    inside = False
    j = n - 1
    for i in range(n):
        if ((poly_y[i] > py) != (poly_y[j] > py)) and \
           (px < (poly_x[j] - poly_x[i]) * (py - poly_y[i]) / (poly_y[j] - poly_y[i]) + poly_x[i]):
            inside = not inside
        j = i
    return inside

@njit
def seg_intersect(ax, ay, bx, by, cx, cy, dx, dy):
    d1 = (dx-cx)*(ay-cy) - (dy-cy)*(ax-cx)
    d2 = (dx-cx)*(by-cy) - (dy-cy)*(bx-cx)
    d3 = (bx-ax)*(cy-ay) - (by-ay)*(cx-ax)
    d4 = (bx-ax)*(dy-ay) - (by-ay)*(dx-ax)
    return ((d1 > 0) != (d2 > 0)) and ((d3 > 0) != (d4 > 0))

@njit
def polygons_overlap(px1, py1, px2, py2):
    n1, n2 = len(px1), len(px2)
    if max(px1) < min(px2) or max(px2) < min(px1):
        return False
    if max(py1) < min(py2) or max(py2) < min(py1):
        return False
    for i in range(n1):
        if pip_numba(px1[i], py1[i], px2, py2):
            return True
    for i in range(n2):
        if pip_numba(px2[i], py2[i], px1, py1):
            return True
    for i in range(n1):
        ni = (i + 1) % n1
        for j in range(n2):
            nj = (j + 1) % n2
            if seg_intersect(px1[i], py1[i], px1[ni], py1[ni],
                           px2[j], py2[j], px2[nj], py2[nj]):
                return True
    return False

def count_overlaps(df, n):
    rows = df[df['id'].str.startswith(f'{n:03d}_')]
    trees = []
    for _, row in rows.iterrows():
        x = float(str(row['x']).replace('s', ''))
        y = float(str(row['y']).replace('s', ''))
        deg = float(str(row['deg']).replace('s', ''))
        trees.append((x, y, deg))
    
    overlaps = 0
    for i in range(len(trees)):
        px1, py1 = get_tree_polygon(*trees[i])
        for j in range(i+1, len(trees)):
            px2, py2 = get_tree_polygon(*trees[j])
            if polygons_overlap(px1, py1, px2, py2):
                overlaps += 1
    return overlaps

print(f"{'N':>4} {'Gap':>10} {'Overlaps':>10}")
print("-"*30)
for n, b, o, gap in gap_by_n[:15]:
    overlaps = count_overlaps(overlap_df, n)
    print(f"{n:>4} {gap:>10.6f} {overlaps:>10}")

In [ ]:
# Key finding: ALL the improvement comes from overlapping configurations
# The question is: Can we find VALID configurations that are better than baseline?

print("\n" + "="*70)
print("CRITICAL FINDING:")
print("="*70)
print("")
print("1. The baseline (70.734327) is the best VALID configuration available")
print("2. The overlapping CSV (67.727) achieves better scores by allowing overlaps")
print("3. ALL 3.0 points of improvement are locked behind overlaps")
print("")
print("The question is: Are there VALID configurations that beat the baseline?")
print("")
print("Options:")
print("1. The baseline IS the global optimum for valid configurations")
print("2. There exist better valid configurations that we haven't found")
print("")
print("Evidence for option 1:")
print("- 9 experiments with various approaches all achieved 70.734327")
print("- 100,000 random attempts per N found no improvement")
print("- C++ optimizer with 100K iterations, 32 restarts found no improvement")
print("")
print("Evidence for option 2:")
print("- The target (68.931058) IS on the leaderboard")
print("- Top teams must have found better valid configurations")
print("- We may be missing a key technique")

In [ ]:
# Let's check what the target score implies
target = 68.931058
baseline = 70.734327
gap_to_target = baseline - target

print(f"\nTarget analysis:")
print(f"Baseline:      {baseline:.6f}")
print(f"Target:        {target:.6f}")
print(f"Gap to target: {gap_to_target:.6f}")
print(f"")
print(f"If the gap is distributed across all N values:")
print(f"  Average improvement per N: {gap_to_target / 200:.6f}")
print(f"")
print(f"If the gap is concentrated in small N (1-32):")
print(f"  Average improvement per N: {gap_to_target / 32:.6f}")
print(f"")
print(f"The overlapping CSV achieves {overlap_total:.6f}")
print(f"This is {target - overlap_total:.6f} BELOW the target")
print(f"")
print(f"This means: Even with overlaps, we're 1.2 points below target!")
print(f"The target IS achievable with valid configurations.")

In [ ]:
# What techniques haven't we tried?
print("\nTechniques we've tried:")
print("="*70)
print("1. Baseline from pre-optimized CSV")
print("2. C++ tree_packer_v21 optimizer")
print("3. Python SA with collision constraints")
print("4. Structured grid packing")
print("5. Overlap repair with SA")
print("6. High-param C++ optimization (20K iter, 10 restarts)")
print("7. SA with translations (jiweiliu approach)")
print("8. Multi-generation C++ (100K iter, 32 restarts)")
print("9. Targeted N optimization with novel approaches")
print("")
print("Techniques we HAVEN'T tried:")
print("="*70)
print("1. Crystalline/lattice packing for large N (>= 58)")
print("2. Asymmetric packing strategies")
print("3. Genetic algorithm with crossover between configurations")
print("4. Beam search with backtracking")
print("5. Different starting configurations (not just baseline)")
print("6. Per-N specialized optimization with different algorithms")
print("7. Ensemble from MULTIPLE external sources (not just local)")
print("8. TPU/GPU-accelerated massive parallel search")
print("9. Mathematical optimization (LP/MIP formulations)")

In [ ]:
# Check if there are any external datasets we haven't explored
print("\nSearching for additional external datasets...")
print("="*70)

# List all CSV files in snapshots
all_csvs = glob.glob('/home/nonroot/snapshots/santa-2025/**/*.csv', recursive=True)
print(f"Total CSV files found: {len(all_csvs)}")

# Score each one
results = []
for fp in all_csvs:
    try:
        df = pd.read_csv(fp)
        if 'id' not in df.columns or 'x' not in df.columns:
            continue
        total = sum(score_n(df, n) for n in range(1, 201))
        results.append((total, fp))
    except:
        pass

results.sort()
print(f"\nTop 10 CSV files by score:")
for score, fp in results[:10]:
    print(f"{score:.6f}  {fp.split('/')[-1]}")